In [1]:
import pandas as pd
import csv 
import stanza
from nltk import tokenize

nlp = stanza.Pipeline(lang='zh', processors='tokenize,pos,lemma,depparse')

2021-08-02 21:56:32 INFO: "zh" is an alias for "zh-hans"
2021-08-02 21:56:32 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsdsimp |
| pos       | gsdsimp |
| lemma     | gsdsimp |
| depparse  | gsdsimp |

2021-08-02 21:56:32 INFO: Use device: cpu
2021-08-02 21:56:32 INFO: Loading: tokenize
2021-08-02 21:56:32 INFO: Loading: pos
2021-08-02 21:56:32 INFO: Loading: lemma
2021-08-02 21:56:32 INFO: Loading: depparse
2021-08-02 21:56:33 INFO: Done loading processors!


In [2]:
cs_data = pd.read_csv("governer_and_preceding_words_extracted.csv")
cs_data.head(10)

,Unnamed: 0,university,sent_id,original_sentence,translation,eng_translation,word_id,first_cs_word_form,first_cs_word_translation,governer_first_cs_word,preceding_word,governer_trans,preceding_trans,preceding_prob,Unnamed: 14,preceding_trans.1,first_cs_word_form.1,Unnamed: 17
0,0.0,CMU,2.0,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,"Rich area, very safe (it belongs to rich area)...",1.0,rich,富人,neighborhood,</s>,社区,</s>,0.000000,NaN,</s>,rich,</s> rich
1,1.0,CMU,4.0,lease 从 八月 开始 ， 具体 日期 可以 商量 。,租约 从 八月 开始 ， 具体 日期 可以 商量 。,"The lease will start in August, the exact date...",1.0,lease,租约,商量,</s>,To discuss,</s>,0.000000,NaN,</s>,lease,</s> lease
2,2.0,CMU,5.0,house 一共 有 两 层 + 地下室 。,房子 一共 有 两 层 + 地下室 。,The house has two floors plus a basement.,1.0,house,房子,有,</s>,There are,</s>,0.000000,NaN,</s>,house,</s> house
3,3.0,CMU,6.0,一共 有 两 个 可以 洗澡 的 bathroom 。,一共 有 两 个 可以 洗澡 的 浴室 。,There are two bathrooms with a bath.,8.0,bathroom,浴室,有,的,There are,The,0.000088,NaN,The,bathroom,The bathroom
4,4.0,CMU,9.0,整个 house 家具 齐全 。,整个 房子 家具 齐全 。,The whole house is fully furnished.,2.0,house,房子,家具,整个,furniture,the whole,0.000078,NaN,the whole,house,the whole house
5,5.0,CMU,10.0,后院 ： deck 。,后院 ： 阳台 。,Backyard: balcony.,3.0,deck,阳台,阳台,：,The balcony,:,0.000006,NaN,:,deck,: deck
6,6.0,CMU,13.0,空调 有 window ac 和 暖气 。,空调 有 窗口 式 冷气机 和 暖气 。,There are window - type air - conditioners and...,3.0,window,窗口,ac,有,空调,There are,0.003321,NaN,There are,window,There are window
7,7.0,CMU,14.0,入住 时间 flexible ， 可 长期 入住 。,入住 时间 灵活 ， 可 长期 入住 。,"Flexible check-in time, can be long stay.",3.0,flexible,灵活,入住,时间,to stay in,time,0.000011,NaN,time,flexible,time flexible
8,8.0,CMU,15.0,感兴趣 的 同学 ， 欢迎 email （ 请 注明 “ 租房 ” ） ， 请 简单 介绍 ...,感兴趣 的 同学 ， 欢迎 发送 邮件 （ 请 注明 “ 租房 ” ） ， 请 简单 介绍 ...,"If you are interested, please send us an email...",6.0,email,发送,欢迎,欢迎,welcome,welcome,0.000165,NaN,welcome,email,welcome email
9,9.0,CMU,2.0,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,"Rich area, very safe (it belongs to rich area)...",1.0,rich,富人,neighborhood,</s>,社区,</s>,0.000000,NaN,</s>,rich,</s> rich


In [10]:
cs_data.shape[0]

1422

In [3]:
#need to match first cs word form in eng_translation (both lower cased) and get the deprel of the word in eng_translation

bools = []
for index, row in cs_data.iterrows():
    #bools.append(str(row['first_cs_word_form']).lower() in str(row['eng_translation']).lower())
    bools.append(str(row['first_cs_word_translation']).lower() in str(row['translation']).lower())
num = bools.count(False)
print(num)
#print(cs_data.iloc[1][7])

0


In [11]:
print(len(bools))

1422


In [12]:
#for chinese data only

cs_sentences = []
for index, row in cs_data.iterrows():
    cs_sentences.append(row['translation'])
    
labels = ['NA'] * len(cs_sentences)
for i in range(len(cs_sentences)):
    with_punct = str(cs_sentences[i])
    sentence = re.sub(r'[^\w\s]', ' ', with_punct)
    length = len(sentence.split())
    doc = nlp(str(sentence))
    for sent in doc.sentences:
        count = 0
        #print(str(cs_data.iloc[i][7]).lower())
        for word in sent.words:
            #print(str(word.text).lower())
            if str(word.text).lower() == str(cs_data.iloc[i][8]).lower() or str(word.text).lower() in str(cs_data.iloc[i][8]).lower():
                label = word.deprel
                labels[i] = label
                break
            count +=1
            if count == length-1 and str(word).lower() != str(cs_data.iloc[i][8]).lower() and str(word.text).lower() not in str(cs_data.iloc[i][8]).lower():
                pass
            #else:
                #labels.append(False)
print(labels[:10])

['compound', 'nsubj', 'nsubj', 'obj', 'nmod', 'compound', 'compound', 'advmod', 'advcl', 'compound']


In [13]:
print(len(labels))

1422


In [14]:
print(labels.count('NA'))

4


In [4]:
cs_sentences = []
for index, row in cs_data.iterrows():
    cs_sentences.append(row['eng_translation'])

In [5]:
tokenize.word_tokenize(cs_sentences[1020])

['Welcome', 'to', 'telephone', 'or', 'email', 'consultation', '.']

In [6]:
print((cs_sentences[1020]))

Welcome to telephone or email consultation.


In [5]:
import re

In [59]:
labels = ['NA'] * len(cs_sentences)
for i in range(len(cs_sentences)):
    with_punct = str(cs_sentences[i])
    sentence = re.sub(r'[^\w\s]', ' ', with_punct)
    length = len(sentence.split())
    doc = nlp(str(sentence))
    for sent in doc.sentences:
        count = 0
        #print(str(cs_data.iloc[i][7]).lower())
        for word in sent.words:
            #print(str(word.text).lower())
            if str(word.text).lower() == str(cs_data.iloc[i][7]).lower() or str(word.text).lower() in str(cs_data.iloc[i][7]).lower():
                label = word.deprel
                labels[i] = label
                break
            count +=1
            if count == length-1 and str(word).lower() != str(cs_data.iloc[i][7]).lower() and str(word.text).lower() not in str(cs_data.iloc[i][7]).lower():
                pass
            #else:
                #labels.append(False)
print(labels[:10])

['amod', 'nsubj', 'nsubj', 'det', 'nsubj:pass', 'NA', 'compound', 'amod', 'obj', 'amod']


In [315]:
with_punct = 'There is an apartment downtown'
sentence = re.sub(r'[^\w\s]', '', with_punct)
print(sentence)
length = len(sentence.split())
doc = nlp(str(sentence))
for sent in doc.sentences:
    count = 0
    for word in sent.words:
        if str(word.text).lower() == 'downtown':
            label = word.deprel
            print(word.text)
            print(label)
            break
        count +=1
        #if count == length-1 and str(word).lower() != str(cs_data.iloc[i][7]).lower() and str(word.text).lower() not in str(cs_data.iloc[i][7]).lower():
            #pass

There is an apartment downtown
downtown
nsubj


In [84]:
print((labels).count('NA'))

496


In [15]:
cs_data['cs_word_annotated_zh'] = labels

In [16]:
cs_data.head()

,Unnamed: 0,university,sent_id,original_sentence,translation,eng_translation,word_id,first_cs_word_form,first_cs_word_translation,governer_first_cs_word,preceding_word,governer_trans,preceding_trans,preceding_prob,Unnamed: 14,preceding_trans.1,first_cs_word_form.1,Unnamed: 17,cs_word_annotated_zh
0,0.0,CMU,2.0,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,"Rich area, very safe (it belongs to rich area)...",1.0,rich,富人,neighborhood,</s>,社区,</s>,0.000000,NaN,</s>,rich,</s> rich,compound
1,1.0,CMU,4.0,lease 从 八月 开始 ， 具体 日期 可以 商量 。,租约 从 八月 开始 ， 具体 日期 可以 商量 。,"The lease will start in August, the exact date...",1.0,lease,租约,商量,</s>,To discuss,</s>,0.000000,NaN,</s>,lease,</s> lease,nsubj
2,2.0,CMU,5.0,house 一共 有 两 层 + 地下室 。,房子 一共 有 两 层 + 地下室 。,The house has two floors plus a basement.,1.0,house,房子,有,</s>,There are,</s>,0.000000,NaN,</s>,house,</s> house,nsubj
3,3.0,CMU,6.0,一共 有 两 个 可以 洗澡 的 bathroom 。,一共 有 两 个 可以 洗澡 的 浴室 。,There are two bathrooms with a bath.,8.0,bathroom,浴室,有,的,There are,The,0.000088,NaN,The,bathroom,The bathroom,obj
4,4.0,CMU,9.0,整个 house 家具 齐全 。,整个 房子 家具 齐全 。,The whole house is fully furnished.,2.0,house,房子,家具,整个,furniture,the whole,0.000078,NaN,the whole,house,the whole house,nmod


In [17]:
cs_data.to_csv('annotated_cs_data_zh.csv', index = False)

In [91]:
import stanza
import pandas as pd
cs_data = pd.read_csv('redoing_cs_data.csv')
cs_data.head()

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

2021-08-04 21:05:14 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-08-04 21:05:14 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2021-08-04 21:05:14 INFO: Use device: cpu
2021-08-04 21:05:14 INFO: Loading: tokenize
2021-08-04 21:05:14 INFO: Loading: pos
2021-08-04 21:05:15 INFO: Loading: lemma
2021-08-04 21:05:15 INFO: Loading: depparse
2021-08-04 21:05:18 INFO: Done loading processors!


In [33]:
trigrams = ['FAILED'] * len(cs_data)
for index, row in cs_data.iterrows():
    doc = nlp(str(row['eng_translation']).lower())
    #print(doc)
    for sent in doc.sentences:
        for word in sent.words:
            #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
            if word.text == str(row['first_cs_word_form']).lower():
                if word.head==0:
                    trigrams[index] = ("root " + word.text)
                else:
                    if word.id < word.head:
                        trigrams[index] = (word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                    else: 
                        trigrams[index] = (word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
                break
                
print(trigrams)


['amod rich neighborhood', 'nsubj lease begin', 'nsubj house has', 'nsubj are bathroom', 'nsubj:pass house furnished', 'dep backyard deck', 'compound window ac', 'amod flexible check', 'root email', 'amod full bathroom', 'root email', 'conj garage driveway', 'amod half bath', 'amod free access', 'root share', 'advmod unbeatable nice', 'compound living room', 'advmod all gone', 'root available', 'nsubj pet discuss', 'obl contact email', 'obj send return', 'nmod delivery view', 'root bathroom', 'FAILED', 'case off parking', 'det no smoking', 'nmod people house', 'nsubj is doorman', 'nsubj lease available', 'root nice', 'nsubj is doorman', 'det an experience', 'root share', 'nsubj neighborhood quiet', 'advmod half furnished', 'advmod fully furnished', 'obl looking house', 'nmod floor house', 'root budget', 'FAILED', 'obl contact email', 'compound leasing office', 'root nice', 'compound email details', 'compound queen bed', 'amod indoor spaces', 'amod young professional', 'FAILED', 'compou

In [34]:
print(len(trigrams))

1476


In [35]:
print(trigrams.count("FAILED"))

121


In [36]:
textfile = open("redone_dep_test.txt", "w")
for element in trigrams:
    textfile.write(element + "\n")
textfile.close()

In [43]:
doc = nlp('Suitable for couples or single.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'couples':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl Suitable couples


In [42]:
doc = nlp('The living room was huge, the walkin closet was huge.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'walkin':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

compound walkin closet


In [44]:
doc = nlp('The whole apartment is shared by two people, so it is easier for them to coordinate.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'shared':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root shared


In [46]:
doc = nlp('The two blocks contain all kinds of food, a bank and a small supermarket.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'blocks':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj blocks contain


In [47]:
doc = nlp('There is plenty of room for friends to come for dinner or parties.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'parties':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

conj dinner parties


In [49]:
doc = nlp('If you need any help, please email us at em@19')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'email':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root email


In [51]:
doc = nlp('Nearby: bars, restaurants and a small supermarket are nearby.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'bars':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj bars nearby


In [52]:
doc = nlp('There are individual houses and individual rooms for rent.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'houses':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj are houses


In [53]:
doc = nlp('Rental time and rent: 7/31/2016 ~ 6/30/2016, 300 / month.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == '300':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nummod 300 month


In [92]:
doc = nlp('There are at least two bedrooms or more.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'bedrooms':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj are bedrooms


In [93]:
doc = nlp('Please email em@26 for details')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'email':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root email


In [94]:
doc = nlp('The landlord covers all utilities except the net.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'utilities':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj covers utilities


In [97]:
doc = nlp('The living room is quiet enough to study, and there\'s a walkin closet to use.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'walkin':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

amod walkin closet


In [98]:
doc = nlp('There is a washing machine in the basement. It can be cleaned within four quarters.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'quarters':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl cleaned quarters


In [99]:
doc = nlp('Toilet: The bathroom is shared with a boy who lives in a small bedroom.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'shared':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root shared


In [100]:
doc = nlp('The price includes sewage, heavy metal, gas, heating and maintece.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'sewage':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj includes sewage


In [101]:
doc = nlp('I also get emails from scammers.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'emails':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj get emails


In [102]:
doc = nlp('Not necessarily two bedrooms.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'bedrooms':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root bedrooms


In [103]:
doc = nlp('There is no such thing as expensive heating in some houses, but still not warm.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'houses':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl is houses


In [104]:
doc = nlp('She knows almost every one of the residents in the apartment, a very nice lady.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'a':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

det a lady


In [105]:
doc = nlp('B: I\'m very responsible for managing a large property.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'managing':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

advcl responsible managing


In [106]:
doc = nlp('The surrounding blocks are inhabited by traditional Jewish families, who live in a harmonious and secure neighborhood.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'blocks':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj:pass blocks inhabited


In [107]:
doc = nlp('Two bedrooms are currently available for rent, yearly lease preferred. ')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'yearly':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

amod yearly lease


In [108]:
doc = nlp('Living room area is quite large, after the table shelf cabinet is still more than enough, suitable for parties. ')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'parties':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl suitable parties


In [109]:
doc = nlp('The first floor of the house is the living room and the kitchen.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'house':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nmod floor house


In [110]:
doc = nlp('The price should be affordable by one person, because it is not shared with others.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'shared':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

advcl affordable shared


In [111]:
doc = nlp('7. Free use of the house basement dryer.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'house':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

compound house dryer


In [112]:
doc = nlp('The rent is 410/month with water and heating (a big savings of money in winter), and you only need to share electricity and Internet.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'money':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nmod savings money


In [113]:
doc = nlp('prefer if you are interested, please contact em@70')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'prefer':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root prefer


In [114]:
doc = nlp('Most of the patterns are obvious and look like frauds.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'patterns':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nmod Most patterns


In [115]:
doc = nlp('The house was kept clean and tidy by the tet, and everyone was quiet. ')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'kept':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root kept


In [116]:
doc = nlp('The price: One bedroom, one living room for two is $1,100.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text == 'price':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root price


In [118]:
doc = nlp('1-3 Bedrooms available.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'bedrooms':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj Bedrooms available


In [121]:
doc = nlp('It has two bedrooms (one large, one small). The kitchen and bathroom are both very large.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'bedrooms':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj has bedrooms


In [126]:
doc = nlp('The whole house has two bedrooms, one bathroom, living room and dining room.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'bedrooms':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj has bedrooms


In [127]:
doc = nlp('As it is my first car, I cherish it very much and dealers do regular maintece.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'dealers':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj dealers do


In [128]:
doc = nlp('Third generation iPad, 32GB RAM, Retina Display.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'ipad':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root iPad


In [129]:
doc = nlp('Students living in houses need heaters very much in winter.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'heaters':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj need heaters


In [130]:
doc = nlp('I posted here.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'posted':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root posted


In [131]:
doc = nlp('Graduation sale')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'sale':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root sale


In [132]:
doc = nlp('As long as it\'s not broken or buggy.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'buggy':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

conj broken buggy


In [133]:
doc = nlp('You can park at the door. It\'s super convenient')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'park':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root park


In [134]:
doc = nlp('Super Security: Gated Apt!')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'gated':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

amod Gated Apt


In [135]:
doc = nlp('There are also some paperwork that needs to be processed in advance.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'paperwork':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj are paperwork


In [138]:
doc = nlp('At least two bedrooms (one man and one woman are both PhDs), or the whole house.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'phds':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

parataxis bedrooms PhDs


In [139]:
doc = nlp('[ Update ] The room is still there')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'update':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root Update


In [140]:
doc = nlp('You can play on the field in your spare time.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'field':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl play field


In [141]:
doc = nlp('There are often activities in the community, such as free snacks, drinks, parties and so on.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'parties':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

conj snacks parties


In [142]:
doc = nlp('Both friends are PhDs, quiet and no party.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'phds':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root PhDs


In [143]:
doc = nlp('State Farm has a different process for recruiting agents than many companies.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'agents':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nmod process agents


In [144]:
doc = nlp('The community provides visitors')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'visitors':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj provides visitors


In [145]:
doc = nlp('Rides to both places are available.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'rides':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj Rides available


In [146]:
doc = nlp('The car is a intermittent problem now.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'intermittent':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

amod intermittent problem


In [147]:
doc = nlp('No other lease will be accepted.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'lease':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj:pass lease accepted


In [148]:
doc = nlp('Miles: about 60,000 miles.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'miles':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root Miles


In [149]:
doc = nlp('Does it bother you to fill in the waiver again?')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'waiver':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj fill waiver


In [150]:
doc = nlp('The room for rent is one of two bedrooms on the second floor.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'bedrooms':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nmod one bedrooms


In [152]:
doc = nlp('The community has its own resident website, all requests can be completed online, or by phone.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'requests':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj:pass requests completed


In [153]:
doc = nlp('No pets, no parties')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'parties':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

parataxis pets parties


In [154]:
doc = nlp('You can email us at em@28')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'email':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root email


In [155]:
doc = nlp('15-18 min walk to the school, then take the campus loop can easily reach any part of the campus.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'min':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

compound min walk


In [156]:
doc = nlp('You can book summer airport rides')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'rides':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj book rides


In [157]:
doc = nlp('There is a shared bathroom on the second floor.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'shared':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

amod shared bathroom


In [158]:
doc = nlp('There is a bathroom above and below the house on two floors.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'house':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

conj bathroom house


In [159]:
doc = nlp('I\'m going home soon. I\'ll transfer my meal plan for ten times.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'meal':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

compound meal plan


In [160]:
doc = nlp('Rent is covered by all utilities')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'utilities':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl covered utilities


In [161]:
doc = nlp('With kitchen and bathroom (shareable), with refrigerator.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'shareable':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

amod kitchen shareable


In [162]:
doc = nlp('Can\'t your mail be delivered?')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'delivered':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root delivered


In [165]:
doc = nlp('Usually quiet, do not have parties, most of the time in the lab.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'parties':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj have parties


In [166]:
doc = nlp('I haven\'t talked to the professor about the project or funding yet.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'professor':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl talked professor


In [167]:
doc = nlp('Now there are two PhDs living there.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'phds':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj are PhDs


In [168]:
doc = nlp('Learners permit, certificate of registration A: I have a driver\'s license, and I have a driver\'s license.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'learners':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj Learners permit


In [170]:
doc = nlp('Yes, I am a Student Team Player!')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'i':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj I Player


In [171]:
doc = nlp('Other utilities are also included, not to mention the Sewer.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'utilities':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj:pass utilities included


In [172]:
doc = nlp('Or is there an apartment nearby where you can park it for free overnight?')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'overnight':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl park overnight


In [175]:
doc = nlp('So even if I could have two permits, I wouldn\'t be able to use my car.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'permits':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj have permits


In [176]:
doc = nlp('prefer a girl.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'prefer':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root prefer


In [177]:
doc = nlp('Seek tennis partner teacher, teach children students if you are interested, please contact em@139')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'seek':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root Seek


In [178]:
doc = nlp('You can Google "Blue Knob" for better dealers nearby. Local dealers in the village are not as good as theirs (personal feeling).')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'dealers':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nmod Google dealers
nsubj dealers good


In [180]:
doc = nlp('I don\'t go to parties.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'parties':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl go parties


In [181]:
doc = nlp('How many miles?')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'miles':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root miles


In [184]:
doc = nlp('I usually prefer to stay at home and do not have parties.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'parties':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj have parties


In [185]:
doc = nlp('Oh, sorry, I have updated my contact information')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'updated':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root updated


In [186]:
doc = nlp('On the first floor of the house are the dining room and the living room and the kitchen. On the second floor are three rooms.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'house':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nmod floor house


In [187]:
doc = nlp('There are many shuttles in front of the school')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'shuttles':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj are shuttles


In [188]:
doc = nlp('Managers are very talkative;')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'managers':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj Managers talkative


In [189]:
doc = nlp('The house is only two blocks away from Frick Park, which is very convenient for exercising in the park.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'blocks':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl:npmod blocks away


In [190]:
doc = nlp('I have been studying social work and working in a city in central America for the past five years')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'social':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

amod social work


In [192]:
doc = nlp('There is plenty of room for friends to come for dinner or parties.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'parties':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj have parties


In [193]:
doc = nlp('-Blair: One apartment and one living room.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'apartment':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

appos Blair apartment


In [195]:
doc = nlp('The small shops and bars above Walnut Street are also good;')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'bars':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

conj shops bars


In [196]:
doc = nlp('The corridor is also cleaned and maintained by management every day')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'management':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obl maintained management


In [197]:
doc = nlp('Please email em@35 for details')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'email':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root email


In [198]:
doc = nlp('There are also campus shuttles.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'shuttles':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj are shuttles


In [199]:
doc = nlp('There are many restaurants in Squirrel Hill.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'restaurants':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj are restaurants


In [200]:
doc = nlp('There are two big masters, a boy and a girl, very easy to get along with.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'masters':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

nsubj are masters


In [202]:
doc = nlp('Permit, no car, but need to take a driver\'s license soon.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'permit':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

root Permit


In [203]:
doc = nlp('Then you can send mass emails to nearby dealers to ask for quotation, and then let them beat each other.')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        #print(word.text + " " + (str(row['first_cs_word_form']).lower()))
        if word.text.lower() == 'emails':
            if word.head==0:
                print("root " + word.text)
            else:
                if word.id < word.head:
                    print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                else: 
                    print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
            break

obj send emails


In [ ]:
Then you can send mass emails to nearby dealers to ask for quotation, and then let them beat each other.

In [204]:
import stanza

nlp = stanza.Pipeline(lang='zh', processors='tokenize,pos,lemma,depparse')
#doc = nlp('Barack Obama was born in Hawaii')
#print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')

2021-08-06 08:35:24 INFO: "zh" is an alias for "zh-hans"
2021-08-06 08:35:24 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsdsimp |
| pos       | gsdsimp |
| lemma     | gsdsimp |
| depparse  | gsdsimp |

2021-08-06 08:35:24 INFO: Use device: cpu
2021-08-06 08:35:24 INFO: Loading: tokenize
2021-08-06 08:35:24 INFO: Loading: pos
2021-08-06 08:35:25 INFO: Loading: lemma
2021-08-06 08:35:25 INFO: Loading: depparse
2021-08-06 08:35:28 INFO: Done loading processors!


In [67]:
trigrams_zh = ['FAILED'] * len(cs_data)
for index, row in cs_data.iterrows():
    doc = nlp(str(row['zh_translation']))
    #print(doc)
    for sent in doc.sentences:
        for word in sent.words:
            #print(word.text == (str(row['first_cs_word_translation'])))
            if word.text == str(row['first_cs_word_translation']):
                #print(word.head)
                if word.head==0:
                    trigrams_zh[index] = ("root " + word.text)
                else:
                    #print(word.id)
                    if word.id < word.head:
                        #print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                        trigrams_zh[index] = (word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                    else: 
                        trigrams_zh[index] = (word.deprel + " " + sent.words[word.head-1].text + " " + word.text)
                break        
#print(trigrams_zh)


In [68]:
print(len(trigrams_zh))
print(trigrams_zh.count("FAILED"))

1476
143


In [69]:
textfile = open("redone_dep_test_zh.txt", "w")
for element in trigrams_zh:
    textfile.write(element + "\n")
textfile.close()

In [350]:
doc = nlp('厕所 对不起 ， 请 问 厕所 在 哪儿 。')
    #print(doc)
for sent in doc.sentences:
    for word in sent.words:
        print(word.text)
        #if word.text == '共用':
            #print(word.head)
        if word.head==0:
            print("root " + word.text)
        else:
            #print(word.id)
            if word.id < word.head:
                #print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
                print(word.deprel + " " + word.text + " " + sent.words[word.head-1].text)
            else: 
                print(word.deprel + " " + sent.words[word.head-1].text + " " + word.text)

厕所
nsubj 厕所 在
对
case 对 不起
不起
xcomp 不起 问
，
punct ， 在
请
aux 请 在
问
advcl 问 在
厕所
obj 问 厕所
在
root 在
哪儿
obj 在 哪儿
。
punct 在 。


In [ ]:
对比 其他 地方 ， 可能 和 很多 人 共用 ， 并且 还 很 贵 ！